In [1]:
import os

In [2]:
%pwd

'c:\\Users\\a12ej56k\\VS code\\Text-Summarizer-Project\\research'

In [3]:

os.chdir('../')

In [4]:
%pwd

'c:\\Users\\a12ej56k\\VS code\\Text-Summarizer-Project'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir : Path
    datapath : Path
    tokenizer_name : Path



In [14]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

In [15]:
class configurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)  
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) ->DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config= DataTransformationConfig(
            root_dir=config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [16]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [17]:
class DataTransformation:
    def __init__(self,config : DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_example_to_features(self,example_batch):
        input_encoding = self.tokenizer(example_batch['dialogue'], max_length =1024 , truncation = True )


        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'],max_length=128, truncation=True)

        return{
            'input_ids' : input_encoding['input_ids'],
            'attention_musk' : input_encoding['attention_musk'],
            'labels' : target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_example_to_features,batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, 'samsum_dataset'))

In [18]:
try:
    config = configurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-01-09 22:08:27,710 : INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-01-09 22:08:27,718 : INFO:common:yaml file: params.yaml loaded successfully]
[2024-01-09 22:08:27,718 : INFO:common:created directory at: artifacts]
[2024-01-09 22:08:27,726 : INFO:common:created directory at: artifacts/data_transformation]


TypeError: __init__() got an unexpected keyword argument 'data_path'